In [ ]:
#load important libaries
import numpy as np
import pandas as pd
import pickle

In [ ]:
#load in the pretrained random forest model
model = pickle.load(open("random_forest_model.sav","rb"))

In [ ]:
#load in the previously saved training data (already processed) and correct formatting issues
training_data = pd.read_csv("processed_train_data.csv")
training_targets = training_data["revenue"]
training_data.drop(["Unnamed: 0","revenue"],axis = 1,inplace = True)


In [ ]:
cols = training_data.columns
print(len(cols))

In [ ]:
#find the feature importances obtained from the random forest fit to determine which variables matter most
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print("Feature importance of %s is %f" %(cols[i],importances[i]))

In [ ]:
top_features = np.array(cols[indices[:10]])
print(top_features)

In [ ]:
#select out the training data corresponding to the top 10 features
train_data = training_data[top_features]
print(train_data.shape)

In [ ]:
print(train_data.head())

In [ ]:
#load in the testing data
testing_data = pd.read_csv("processed_test_data.csv")
testing_data.drop("Unnamed: 0",axis = 1,inplace = True)

In [ ]:
print(testing_data.shape)
print(testing_data.head())

In [ ]:
#select out the testing data corresponding to the top 10 features
test_data = testing_data[top_features]
print(test_data.shape)

In [ ]:
print(test_data.head())

In [ ]:
#train another random forest model on the limited dataset (only 10 features)
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 1000,n_jobs = -1)
fit = model.fit(train_data,training_targets)
predictions = model.predict(test_data)

In [ ]:
#save the model for future use
import pickle
filename = "random_forest_selected.sav"
pickle.dump(model,open(filename,"wb"))

In [ ]:
#wave predictions
final = pd.DataFrame(predictions,columns = ["Revenue"])
print(final.head())

In [ ]:
#write predictions to file
final.to_csv("select_features_prediction.csv")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#implement basic neural network for regression
num_variables = len(top_features)

def basic_model():
    nn_model = Sequential()
    nn_model.add(Dense(num_variables,input_dim = num_variables,activation = "relu"))
    nn_model.add(Dense(1))
    nn_model.compile(loss = "mean_squared_error",optimizer = "adam")
    return nn_model

In [ ]:
print(train_data.shape)
print(training_targets.shape)